In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random


In [2]:
headers = {
    "User-Agent": "MI-Campaign-Finance-Research/1.0 (contact: jwh1097@gmail.com)"
}

base_url = "https://campaignfinance.us/ottawa.mi/"

id_dict = {}
link_dict = {}

committee_id = 0
internal_id = 0

search_url = f"https://campaignfinance.us/ottawa.mi/iCommitteeSearchProcess.php?strSearchBy=1&iCommitteeVisibleID={committee_id}&cmdSubmit=Search"
documents_url = f"https://campaignfinance.us/ottawa.mi/iDocuments.php?iCommitteeID={internal_id}&cmdOk=View+Campaign+Statements"


In [3]:
post_2024 = pd.read_csv("Post-General_2024_CF_Submissions.csv")
committee_ids = post_2024["Committee ID"]
committee_ids = pd.read_csv("test.csv")["committee_id"] # 5 committee ids for testing

## Get the internal Campain Finance committee \#

In [4]:
for id in committee_ids:
    
    committee_id = id

    search_url = f"https://campaignfinance.us/ottawa.mi/iCommitteeSearchProcess.php?strSearchBy=1&iCommitteeVisibleID={committee_id}&cmdSubmit=Search"

    r = requests.get(search_url, headers=headers) # comment this line out while making edits to avoid sending too many requests to CF US
    soup = BeautifulSoup(r.text, 'html.parser')

    # get a link to each committee and save internal id
    for link in soup.find_all('a'):
        x =  link.get('href')
        if x[:-3] == "iCommitteePortal.php?iCommitteeID=":
            link_dict[committee_id] = base_url + x
            id_dict[committee_id] = int(x[-3:])

    # A brief pause out of respect
    time.sleep(random.uniform(1.0, 2.0))

In [6]:
link_dict

{97221: 'https://campaignfinance.us/ottawa.mi/iCommitteePortal.php?iCommitteeID=417',
 96945: 'https://campaignfinance.us/ottawa.mi/iCommitteePortal.php?iCommitteeID=241',
 96961: 'https://campaignfinance.us/ottawa.mi/iCommitteePortal.php?iCommitteeID=223',
 96976: 'https://campaignfinance.us/ottawa.mi/iCommitteePortal.php?iCommitteeID=160',
 97019: 'https://campaignfinance.us/ottawa.mi/iCommitteePortal.php?iCommitteeID=208'}

In [7]:
id_dict

{97221: 417, 96945: 241, 96961: 223, 96976: 160, 97019: 208}

## Get list of Documents for each committee

In [27]:
df = pd.DataFrame(columns= ["committee_id", "document", "received", "due", "pages", "pdf_url"])
df.head()

,committee_id,document,received,due,pages,pdf_url


In [ ]:
for id in id_dict:
    internal_id = id_dict[id]
    documents_url = f"https://campaignfinance.us/ottawa.mi/iDocuments.php?iCommitteeID={internal_id}&cmdOk=View+Campaign+Statements"
    
    resp = requests.get(documents_url)

    # A brief pause out of respect
    time.sleep(random.uniform(0.2, 1.0))

    soup = BeautifulSoup(resp.text, 'html.parser')

    table = soup.find("table", class_="DataList")

    records = []

    for row in table.find_all("tr")[1:]:
        cells = row.find_all("td")
        if len(cells) < 5:
            continue

        link = cells[4].find("a")

        record = {
            "committee_id": id,
            "document": cells[0].get_text(strip=True),
            "received": cells[1].get_text(strip=True),
            "due": cells[2].get_text(strip=True),
            "pages": cells[3].get_text(strip=True),
            "pdf_url": "https://campaignfinance.us" + link["href"] if link else None
        }
        if record["pdf_url"] == None:
            continue

        records.append(record)
    
    df = pd.concat([df, pd.DataFrame(records)], ignore_index=True)
    pd.set_option("display.max_colwidth", None)

In [29]:
df.to_csv("output.csv")